# Connection Examples

## Connecting to a default Redis instance, running locally.

In [2]:
import redis

connection = redis.Redis()
connection.ping()

True

### by default Redis return binary responses, to decode them use decode_responses=True

In [3]:
import redis

decoded_connection = redis.Redis(decode_responses=True)
decoded_connection.ping()

True

## Connecting to a redis instance, specifying a host and port with credentials.

In [4]:
import redis

user_connection = redis.Redis(host='localhost', port=6380, username='dvora', password='redis', decode_responses=True)
user_connection.ping()

True

## Connecting to Redis instances by specifying a URL scheme.
Parameters are passed to the following schems, as parameters to the url scheme.

Three URL schemes are supported:

- `redis://` creates a TCP socket connection. <https://www.iana.org/assignments/uri-schemes/prov/redis>
- `rediss://` creates a SSL wrapped TCP socket connection. <https://www.iana.org/assignments/uri-schemes/prov/rediss>
- ``unix://``: creates a Unix Domain Socket connection.


In [7]:
url_connection = redis.from_url("redis://localhost:6379?decode_responses=True&health_check_interval=2")

url_connection.ping()

True

## Connecting to a Sentinel instance

In [ ]:
from redis.sentinel import Sentinel
sentinel = Sentinel([('localhost', 26379)], socket_timeout=0.1)
sentinel.discover_master("redis-py-test")